## Robust Regression - Julia

In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [2]:
# Import data, already split (in prediction)
X_train = CSV.File("../clean_data/X_train.csv") |> Tables.matrix
X_test = CSV.File("../clean_data/X_test.csv") |> Tables.matrix
y_train = CSV.File("../clean_data/y_train.csv") |> Tables.matrix
y_test = CSV.File("../clean_data/y_test.csv") |> Tables.matrix

X_train_normalized = CSV.File("../clean_data/X_train_normalized.csv") |> Tables.matrix
X_test_normalized = CSV.File("../clean_data/X_test_normalized.csv") |> Tables.matrix

# Remove index column
X_train = X_train[:, 2:end]
X_test = X_test[:, 2:end]
y_train = y_train[:, 2:end]
y_test = y_test[:, 2:end]

X_train_normalized = X_train_normalized[:, 2:end]
X_test_normalized = X_test_normalized[:, 2:end]

7035×55 Matrix{Float64}:
 -0.197732   -0.527299   1.80136   …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.173662   -0.523402   1.98385      0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.146139    2.33067    1.80136      0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.271525    2.35502    1.98385      0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.287863   -0.541784   1.80136      0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.286081   -0.532651   1.98385   …  0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.218862   -0.5754     1.80136      0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.35894    -0.561301   1.98385      0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.228299    0.180355   1.80136      0.0  1.0  0.0  0.0  0.0  0.0  0.0
  0.406002    0.227634   1.98385      0.0  1.0  0.0  0.0  0.0  0.0  0.0
 -0.211995   -0.527299   1.80136   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.208935   -0.523402   1.98385      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.416753   -0.527299   1.80136      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋮                                ⋱   

## LASSO

In [3]:
function solve_lasso(p, n, X, y, lambda)
    
    # Create a JuMP model
    model = Model(Gurobi.Optimizer)
        
    # Insert variables
    @variable(model, beta[j=1:p])
    @variable(model, a[j=1:p]>=0)
    @variable(model, t>=0)
    
    # Insert constraints
    # Linearize norm 1
    @constraint(model,[j=1:p], beta[j]<=a[j])
    @constraint(model,[j=1:p], -beta[j]<=a[j])
    
    # The residual term
    @constraint(model, sum((y[i]-sum(X[i,j]*beta[j] for j=1:p))^2 for i=1:n) <= t)
    
    # Objective
    @objective(model, Min, t + lambda*sum(a[j] for j=1:p))
        
    # Solve the optimization problem
    optimize!(model)
    
    # Retrieve the objective value
    objective = JuMP.objective_value(model)
    beta = JuMP.value.(beta)
    
    return objective, beta
end

solve_lasso (generic function with 1 method)

### Non-normalized

In [4]:
# Function to calculate R^2
function calculate_r_squared(y_test, X_test, beta)
    SS_res = sum((y_test[i] - sum(X_test[i, j] * beta[j] for j = 1:size(X_test, 2)))^2 for i = 1:size(y_test, 1))
    y_test_mean = mean(y_test)
    SS_tot = sum((y_test[i] - y_test_mean)^2 for i = 1:size(y_test, 1))
    return 1 - SS_res / SS_tot
end

# Search for the best lambda
best_lambda = -1
best_r_squared = -Inf

n_train, p_train = size(X_train)

for lambda in 0:0.01:5  # Adjust the range and step as needed
    obj_value, beta = solve_lasso(p_train, n_train, X_train, y_train, lambda);
    r_squared = calculate_r_squared(y_test, X_test, beta)

    println("Lambda = $lambda, R^2 = $r_squared")

    if r_squared > best_r_squared
        best_r_squared = r_squared
        best_lambda = lambda
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x9d4bf0b2
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 110 rows and 55 columns
Presolve time: 0.00s
Presolved: 52 rows, 108 columns, 1006 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 1.326e+03
 Factor NZ  

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x19822c3c
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [2e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0xe54070cc
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [3e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x69c7f42b
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [4e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0xd1dafa80
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [5e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

...

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x5edeefa5
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [5e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x82b4598a
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [5e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x654880a6
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e+01, 1e+16]
  QLMatrix range   [1e+00, 5e+11]
  Objective range  [5e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Excessive output truncated after 524803 bytes.

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [5]:
println("Best lambda: $best_lambda with R^2: $best_r_squared")

Best lambda: 4.19 with R^2: 0.11451770932553262


### Normalized

In [6]:
# Search for the best lambda
best_lambda = -1
best_r_squared = -Inf

n_train, p_train = size(X_train_normalized)

for lambda in 0:0.01:5  # Adjust the range and step as needed
    obj_value, beta = solve_lasso(p_train, n_train, X_train_normalized, y_train, lambda);
    r_squared = calculate_r_squared(y_test, X_test_normalized, beta)

    println("Lambda = $lambda, R^2 = $r_squared")

    if r_squared > best_r_squared
        best_r_squared = r_squared
        best_lambda = lambda
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x5a0d4c4e
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+13]
  QLMatrix range   [1e+00, 2e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 110 rows and 55 columns
Presolve time: 0.00s
Presolved: 52 rows, 108 columns, 1006 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 1.326e+03
 Factor NZ  

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0xb05c9e80
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+13]
  QLMatrix range   [1e+00, 2e+10]
  Objective range  [2e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x54e9904c
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+13]
  QLMatrix range   [1e+00, 2e+10]
  Objective range  [3e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0x62553bf4
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+13]
  QLMatrix range   [1e+00, 2e+10]
  Objective range  [5e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 111 columns and 220 nonzeros
Model fingerprint: 0xbf1f000b
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+13]
  QLMatrix range   [1e+00, 2e+10]
  Objective range  [7e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+08, 2e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.00s
Presolved: 162 rows, 163 columns, 1226 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 55
 AA' NZ     : 3.285e+03
 Factor NZ  : 3.859e+03
 Factor Ops : 1.649e+05 (les

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [7]:
println("Best lambda: $best_lambda with R^2: $best_r_squared")

Best lambda: 0.06 with R^2: 0.11446226869497755
